# Bilan hydrique à partir d'observations Météo-France horaires pour les dernières 24 h

## Reconstruction des données météorologiques pour une station de référence

### Définition des paramètres

In [1]:
# Définition de la station de référence
REF_STATION_NAME = 'La Petite Claye'
REF_STATION_LATLON = [48.541356, -1.615400]
# Altitude (m)
REF_STATION_ALTITUDE = 50.

# Identification de l'API Météo-France
APPLICATION_ID = 'ZlFGb1VCNzdlQ3c5QmhSMU1IbE8xQTluOE0wYTpUS3l1YkcweGJmSTJrQlJVaGNiSkNHTXczdHNh'

# Météo-France API
METEOFRANCE_API = 'DPPaquetObs'

# Fréquence des données climatiques
METEOFRANCE_FREQUENCE = 'horaire'

NN_NOMBRE = None
# NN_RAYON_KM = 20.
NN_RAYON_KM = 35.

# Liste des variables météorologiques utilisées dans les calculs d'ETP et de bilan hydrique
LISTE_VARIABLES_METEO = ['vitesse_vent_10m', 'temperature_2m', 'humidite_relative',
                         'rayonnement_global', 'precipitation']

# Fonction pour aggrégé chaque variable météorologique
VARIABLE_AGGREGATORS = {
    'vitesse_vent_10m': lambda x: x.mean(0),
    'temperature_2m': lambda x: x.mean(0),
    'humidite_relative': lambda x: x.mean(0),
    'rayonnement_global': lambda x: x.sum(0),
    'precipitation': lambda x: x.sum(0),
    'etp': lambda x: x.sum(0),
}

### Lecture de la liste des stations

In [2]:
import meteofrance

# Initialisation d'un client pour accéder à l'API Météo-France
client = meteofrance.Client(METEOFRANCE_API, application_id=APPLICATION_ID)

# Issue a series of API requests an example. For use this test, you must first subscribe to the arome api with your application
client.session.headers.update({'Accept': '*/*'})

In [3]:
import pandas as pd

LECTURE_LISTE_STATIONS = True

filepath_liste_stations = meteofrance.get_filepath_liste_stations(client)

if LECTURE_LISTE_STATIONS:
    # Lecture de la liste des stations
    df_liste_stations = pd.read_csv(
        filepath_liste_stations, index_col=client.id_station_label)
else:
    # Demande de la liste des stations
    section = meteofrance.SECTION_LISTE_STATIONS
    response = meteofrance.demande(client, section)
    df_liste_stations = meteofrance.response_text_to_frame(
        client, response, index_col=client.id_station_label)

    # Sauvegarde de la liste des stations
    df_liste_stations.to_csv(filepath_liste_stations)

### Sélection des plus proches voisins de la station de référence

In [4]:
import geo

df_liste_stations_nn = geo.selection_plus_proches_voisins(
    df_liste_stations, REF_STATION_LATLON, client.latlon_labels,
    nombre=NN_NOMBRE, rayon_km=NN_RAYON_KM)

df_liste_stations_nn

,Id_omm,Nom_usuel,Latitude,Longitude,Altitude,Date_ouverture,Pack,distance
Id_station,,,,,,,,
35044001,NaN,BROUALAN,48.485667,-1.640833,99,1976-01-01,ETENDU,6
50410003,NaN,PONTORSON,48.585667,-1.505167,33,1997-03-01,RADOME,9
35224001,NaN,PLERGUER,48.524833,-1.843667,30,1989-07-01,ETENDU,17
35110003,NaN,FEINS SA,48.326833,-1.596833,87,2005-04-29,RADOME,24
35225001,NaN,PLESDER,48.406833,-1.924833,56,1969-04-01,ETENDU,27
35178001,NaN,MEZIERES-SUR-C.,48.308833,-1.439000,71,1950-06-01,ETENDU,29
50531001,NaN,ST OVIN,48.682500,-1.248667,155,1970-06-01,ETENDU,31
50218001,7133.0,GRANVILLE,48.834500,-1.613667,37,1921-01-01,ETENDU,33
35228001,7125.0,DINARD,48.584833,-2.076333,65,1948-03-29,RADOME,34


### Obtention des données météorologiques pour les stations voisines

In [5]:
LECTURE_DONNEE = True

if LECTURE_DONNEE:
    # Lecture des données des stations
    DATE_DEB_PERIODE = '2025-01-12T10:00:00Z'
    DATE_FIN_PERIODE = '2025-01-13T09:00:00Z'
    filepath_donnee = meteofrance.get_filepath_donnee_periode(
        client, DATE_DEB_PERIODE, DATE_FIN_PERIODE,
        df_liste_stations=df_liste_stations_nn)
    df_meteo = pd.read_csv(
        filepath_donnee, parse_dates=[client.time_label],
        index_col=[client.id_station_donnee_label, client.time_label])
else:
    # Demande des données des stations
    variables = [client.variables_labels[METEOFRANCE_FREQUENCE][k]
                 for k in LISTE_VARIABLES_METEO]
    df_meteo = meteofrance.compiler_donnee_des_departements(
        client, df_liste_stations_nn,
        frequence=METEOFRANCE_FREQUENCE)[variables]
    
    # Sauvegarde des données des stations
    time = df_meteo.index.to_frame()[client.time_label]
    date_deb_periode = time.min().isoformat().replace("+00:00", "Z")
    date_fin_periode = time.max().isoformat().replace("+00:00", "Z")
    filepath_donnee = meteofrance.get_filepath_donnee_periode(
        client, date_deb_periode, date_fin_periode,
        df_liste_stations=df_liste_stations_nn)
    df_meteo.to_csv(filepath_donnee)

### Interpolation des données météorologiques à la station de référence

Les variables sont également renommées en utilisant des noms communs à l'ensemble de ces notebooks quelque soit l'API utilisée.

In [6]:
LECTURE_DONNEE_REF = True

str_ref_station_name = REF_STATION_NAME.lower().replace(' ', '')
filepath_donnee_ref = filepath_donnee.with_name(
    filepath_donnee.stem + '_' + str_ref_station_name + filepath_donnee.suffix)

if LECTURE_DONNEE_REF:
    # Lecture des données des stations pour la période
    df_meteo_ref_heure = pd.read_csv(
        filepath_donnee_ref, parse_dates=[client.time_label],
        index_col=client.time_label)
else:
    # Demande des données des stations
    df_meteo_ref_heure = geo.interpolation_inverse_distance_carre(
        df_meteo, df_liste_stations_nn['distance'])
    
    # Sauvegarde par département
    df_meteo_ref_heure.to_csv(filepath_donnee_ref)

df_meteo_ref_heure = meteofrance.renommer_variables(
    client, df_meteo_ref_heure, METEOFRANCE_FREQUENCE)

### Conversion des unités

In [7]:
df_meteo_ref_heure_si = meteofrance.convertir_unites(
    client, df_meteo_ref_heure)

### Estimation de l'ETP journalière pour la station de référence

####  Calcul de l'ETP horaire à partir des données météorologiques

In [8]:
from etp import calcul_etp

df_meteo_ref_heure_si['etp'] = calcul_etp(
    df_meteo_ref_heure_si, *REF_STATION_LATLON, REF_STATION_ALTITUDE)

df_meteo_ref_heure_si

,vitesse_vent_10m,temperature_2m,humidite_relative,rayonnement_global,precipitation,etp
validity_time,,,,,,
2025-01-13 09:00:00+00:00,1.802227,270.882297,0.980004,306000.0,0.000000,0.000937
2025-01-13 08:00:00+00:00,1.703129,269.611870,0.980583,0.0,0.000000,0.000815
2025-01-13 07:00:00+00:00,2.528708,269.339300,0.978264,0.0,0.000000,0.001205
2025-01-13 06:00:00+00:00,2.038824,268.906707,0.979421,0.0,0.000000,0.000953
2025-01-13 05:00:00+00:00,3.497317,269.099133,0.971162,0.0,0.000000,0.001957
2025-01-13 04:00:00+00:00,1.005537,269.016482,0.987681,0.0,0.000000,0.000325
2025-01-13 03:00:00+00:00,0.947239,269.013513,0.988264,0.0,0.000000,0.000294
2025-01-13 02:00:00+00:00,1.753183,269.311281,0.987681,0.0,0.000000,0.000520
2025-01-13 01:00:00+00:00,0.816642,269.928511,0.997681,0.0,0.000000,0.000054


#### Aggrégation à l'échelle journalière

In [9]:
# Calcul des valeurs journalières des variables météo
s_meteo_ref_si = pd.Series(dtype=float)
for variable, series in df_meteo_ref_heure_si.items():
    s_meteo_ref_si.loc[variable] = VARIABLE_AGGREGATORS[variable](
        df_meteo_ref_heure_si[variable])

## Estimation du bilan hydrique

### Définition des paramètres

Sources :

- Coefficients culturaux :
  - [ARDEPI](https://www.ardepi.fr/nos-services/vous-etes-irrigant/estimer-ses-besoins-en-eau/maraichage/)
  - [Chambre d’agriculture Nouvelle-Aquitaine](https://gironde.chambre-agriculture.fr/fileadmin/user_upload/Nouvelle-Aquitaine/100_Inst-Gironde/Documents/pdf_grandes-cultures_accompagnement-technique_mieux-irriguer/Messages_irrigation_2019/message_1/Tableau_Coefficients_Culturaux_Kc_.02.pdf)


In [10]:
# RFU finale cible (mm)
RFU_CIBLE = None

# Fraction du sol occupé par des cailloux et graviers (entre 0 pour absence de cailloux et 1 pour totalité de cailloux)
FRACTION_CAILLOUX = 0.1

# Choix de la texture
TEXTURE = 'Terres limoneuses'

# Coefficient de conversion de la RU en RFU (entre 1/2 et 2/3)
RU_VERS_RFU = 2. / 3

# Fraction de la réserve utile du sol remplie d'eau (entre 0 pour une période sèche et 1 pour une période pluvieuse)
FRACTION_REMPLIE = 1.

# Besoin d'irrigation minimal à partir du quel irriguer (mm)
SEUIL_IRRIGATION = 0.1

# Conversion de hauteur (mm) vers durée d'irrigation (min)
HAUTEUR_VERS_DUREE_IRRIGATION = 10

### Plot du bilan

In [14]:
import bilan
import numpy as np
import panel as pn
import plotly.graph_objects as go

pn.extension('plotly')


def creer_plot_sol(s, width=500, height=400):
    idx_deb = 1
    idx_fin = 5
    x = s.index[idx_deb:]
    s_ru = s.iloc[1:5].astype(float).values
    y = np.concatenate([[s_ru[0]], s_ru[1:] - s_ru[:-1]])
    measure = ['absolute'] + ['delta'] * (idx_fin - idx_deb - 1)
    wf = go.Waterfall(x=x, y=y, measure=measure,
                      texttemplate='%{final:.1f}', cliponaxis=False)
    fig = go.Figure(wf)
    fig.update_layout(
        title="Réserve accessible aux racines (valeurs absolues)",
        yaxis_title="Hauteur (mm)",
        width=width,
        height=height
    )
    plotly_pane = pn.pane.Plotly(fig)
    
    return plotly_pane

def creer_plot_besoin(s, width=500, height=400):
    idx_deb = 4
    idx_fin = 9
    x = s.index[idx_deb:]
    y = s.iloc[idx_deb:idx_fin].astype(float)
    measure = ['absolute'] + ['relative'] * (idx_fin - idx_deb - 2) + ['absolute']
    wf = go.Waterfall(x=x, y=y, measure=measure,
                      texttemplate='%{delta:.1f}', cliponaxis=False)
    fig = go.Figure(wf)
    fig.update_layout(
        title="Bilan hydrique (différences)",
        yaxis_title="Hauteur (mm)",
        width=width,
        height=height
    )
    plotly_pane = pn.pane.Plotly(fig)

    return plotly_pane

# Fonction pour plot intéractif
def creer_plots(hauteur_vers_duree_irrigation, culture, stade):
    # Get the data
    s_bilan = bilan.calcul_bilan(
        TEXTURE, FRACTION_CAILLOUX,
        culture, stade,
        s_meteo_ref_si,
        rfu_cible=RFU_CIBLE, fraction_remplie=FRACTION_REMPLIE, ru_vers_rfu=RU_VERS_RFU,
        seuil_irrigation=SEUIL_IRRIGATION,
        hauteur_vers_duree_irrigation=hauteur_vers_duree_irrigation)  

    plot_sol = creer_plot_sol(s_bilan)
    plot_besoin = creer_plot_besoin(s_bilan)
    plot_titre = pn.pane.Markdown(
        f"## Pour {culture.lower()} au stade {stade.lower()}")

    p = pn.Column(plot_titre, pn.Row(plot_sol, plot_besoin))
    
    if s_bilan['irrigation']:
        plot_irrigation = pn.pane.Markdown(
            f"### Besoin d'arroser {s_bilan['duree_irrigation']:.0f} min")
        p = pn.Column(p, plot_irrigation)
    
    return p

# Panel widgets for interaction
hauteur_vers_duree_irrigation_widget = pn.widgets.EditableIntSlider(
    name="Hauteur d'eau par durée d'irrigation (mm min-1)", 
    start=1, end=180, step=1, value=HAUTEUR_VERS_DUREE_IRRIGATION)
list_kc = list(bilan.KC)
culture_widget = pn.widgets.Select(
    name='Culture', options=list_kc, value=list_kc[0])
stade_widget = pn.widgets.Select(
    name='Stade', options=list(bilan.KC[list_kc[0]]))

# Update the options of stade_widget based on culture_widget
def update_stade_options(event):
    selected_culture = event.new
    stade_widget.options = list(bilan.KC[selected_culture])
    stade_widget.value = list(bilan.KC[selected_culture])[0]  # Set to the first available stage

culture_widget.param.watch(update_stade_options, 'value')

# Interactive plot
@pn.depends(
    hauteur_vers_duree_irrigation_widget,
    culture_widget, stade_widget
)
def update_plot(
    hauteur_vers_duree_irrigation,
    culture, stade
):
    return creer_plots(
        hauteur_vers_duree_irrigation,
        culture, stade
    )

In [15]:
# Layout
dashboard = pn.Column(
    hauteur_vers_duree_irrigation_widget,
    pn.Row(culture_widget, stade_widget),
    update_plot
)

In [16]:
dashboard.servable()

Column
    [0] EditableIntSlider(end=180, name="Hauteur d'eau p..., start=1, value=10, width=300)
    [1] Row
        [0] Select(name='Culture', options=['Ail', 'Artichaut', ...], value='Ail')
        [1] Select(name='Stade', options=['De la fin de la levée à ...], value='De la fin de l...)
    [2] ParamFunction(function, _pane=Column, defer_load=False)